In [1]:
# from transformers import BertTokenizer, BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, BertForMaskedLM
from transformers import pipeline
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
import torch

[nltk_data] Downloading package wordnet to /Users/gao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/gao/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
# parameters
top_k = 10
eval_taxo_path = "taxos/HiExpan1.txt"
use_queries = "custom_queries/default_queries.txt"

In [31]:
query_templates = []
column_names = []

with open(use_queries, "r") as fin:
    for line in fin.readlines():
        if not line.startswith("#"):
            query, name = line.split(",")
            query_templates.append(query.strip())
            column_names.append(name.strip())

print(list(zip(query_templates, column_names)))

[('{token} [MASK]', '{token} [MASK]'), ('[MASK] {token}', '[MASK] {token}'), ('{token} is a [MASK]', 'is a'), ('{token} is an [MASK]', 'is an'), ('{token} is a kind of [MASK]', 'kind of'), ('{token} is a type of [MASK]', 'type of'), ('{token} is an example of [MASK]', 'example of'), ('[MASK] such as {token}', 'such as'), ('A [MASK] such as {token}', 'a such as'), ('An [MASK] such as {token}', 'an such as'), ('My favorite [MASK] is {token}', 'favorite')]


In [15]:
eval_parents = []
eval_children = []

with open(eval_taxo_path , "r") as fin:
    for concept_pair in fin.readlines():
        parent, child = concept_pair.split(',')
        eval_parents.append(parent.strip())
        eval_children.append(child.strip())

In [3]:
# instantiate a standard bert-base tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# instantiate another tokenizer for adding customized tokens
tokenizer_extended = BertTokenizer.from_pretrained('bert-base-uncased')

not_in_bert_parent_lemmas = []

with open("model_configuration/add_tokenizer_vocabulary.txt", "r") as fin:
    for word in fin.readlines():
        not_in_bert_parent_lemmas.append(word.strip())

tokenizer_extended.add_tokens(not_in_bert_parent_lemmas)

31

In [7]:
# can also use mps on mac
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device('mps')
else:
    torch.device('cpu')

# # load models
# model1a = BertForMaskedLM.from_pretrained('/content/drive/MyDrive/mlm_final_exps/entity_masking_train_test/checkpoint-117350')
# model1a.to(device)
# model1a.eval()
#
# model1b = BertForMaskedLM.from_pretrained('/content/drive/MyDrive/mlm_final_exps/entity_masking_train_test_only_one/final/checkpoint-58675')
# model1b.to(device)
# model1b.eval()
#
# model2a = BertForMaskedLM.from_pretrained('/content/drive/MyDrive/mlm_final_exps/random_masking_train_test/final/checkpoint-58675')
# model2a.to(device)
# model2a.eval()
#
# model2b = BertForMaskedLM.from_pretrained('/content/drive/MyDrive/mlm_final_exps/random_masking_train_only/checkpoint-82146')
# model2b.to(device)
# model2b.eval()
#
# model0a = BertForMaskedLM.from_pretrained('/content/drive/MyDrive/exp/yelp/out_random_masking-bert/checkpoint-80000')
# model0a.to(device)
# model0a.eval()
#
# model0b = DistilBertForMaskedLM.from_pretrained('/content/drive/MyDrive/exp/yelp/out-entity-masking/checkpoint-73675')
# model0b.to(device)
# model0b.eval()

In [8]:
# or, just use pipeline
# !pip install transformers
# load a single model or multiple
#
# unmasker1a = pipeline('fill-mask', tokenizer=tokenizer_extended, model=model1a, device=device)
# unmasker1b = pipeline('fill-mask', tokenizer=tokenizer_extended, model=model1b, device=device)
# unmasker2a = pipeline('fill-mask', tokenizer=tokenizer_extended, model=model2a, device=device)
# unmasker2b = pipeline('fill-mask', tokenizer=tokenizer_extended, model=model2b, device=device)
#
# unmasker0a = pipeline('fill-mask', tokenizer='bert-base-uncased', model=model0a, device=device)
# unmasker0b = pipeline('fill-mask', tokenizer='distilbert-base-uncased', model=model0b, device=device)

# whole_word_unmasker = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking', device=device)
base_unmasker = pipeline('fill-mask', model='bert-base-uncased', device=device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
lemmatizer = WordNetLemmatizer()

def lemmatize_a_word(word):
    split_word = word.split()
    # print(split_word)
    ngram=len(split_word)
    if ngram == 1:
        return lemmatizer.lemmatize(word, pos='n')
    else:
        last_word =split_word[-1]
        lem = lemmatizer.lemmatize(last_word, pos='n')
        new_word = ' '.join(split_word[:-1] + [lem])
        return new_word

# Customisable RaTE patterns
(add more descriptions)

In [18]:
# paper page 5
def generate_queries_paper(token):
    token = lemmatize_a_word(token) # lemmatize to remove inflection

    p1a = f" {token} [MASK] ."
    p1b = f" [MASK] {token} ."

    p2a = f" {token} is a [MASK] ."
    p2b = f" {token} is an [MASK] ."

    p3a = f" {token} is a kind of [MASK] ."
    p3b = f" {token} is a type of [MASK] ."
    p3c = f" {token} is an example of [MASK] ."

    p4a = f" [MASK] such as {token} ."
    p4b = f" A [MASK] such as {token} ."
    p4c = f" An [MASK] such as {token} ."

    p5a = f" My favorite [MASK] is {token} ."

    # read customized patterns from filestream?
    # ...

    return [p1a, p1b,
            p2a, p2b,
            p3a, p3b, p3c,
            p4a, p4b, p4c,
            p5a]

In [36]:
def generate_queries_from_template(token):
    test_queries = []

    for template in query_templates:
        test_queries.append(template.replace("{token}", token) + " .")

    return test_queries

In [45]:
from tqdm import tqdm

for predictor in [base_unmasker]:
    predictions = {}

    # get 1e results
    for child in tqdm(eval_children):
        queries = generate_queries_from_template(child)
        queries_unmasked = base_unmasker(queries, top_k=top_k)
        for i, column in enumerate(column_names):
            if child not in predictions:
                predictions[child] = {}
            predictions[child][column] = queries_unmasked[i]

100%|██████████| 524/524 [01:40<00:00,  5.22it/s]


In [47]:
predictions["mussel"]['is a']

[{'score': 0.10947363078594208,
  'token': 14638,
  'token_str': 'perennial',
  'sequence': 'mussel is a perennial.'},
 {'score': 0.0480949766933918,
  'token': 17901,
  'token_str': 'weed',
  'sequence': 'mussel is a weed.'},
 {'score': 0.02520201914012432,
  'token': 23566,
  'token_str': 'vegetarian',
  'sequence': 'mussel is a vegetarian.'},
 {'score': 0.02241355925798416,
  'token': 24024,
  'token_str': 'bacterium',
  'sequence': 'mussel is a bacterium.'},
 {'score': 0.022051049396395683,
  'token': 26835,
  'token_str': 'pathogen',
  'sequence': 'mussel is a pathogen.'},
 {'score': 0.01925797015428543,
  'token': 15267,
  'token_str': 'predator',
  'sequence': 'mussel is a predator.'},
 {'score': 0.019203638657927513,
  'token': 25742,
  'token_str': 'sponge',
  'sequence': 'mussel is a sponge.'},
 {'score': 0.017182761803269386,
  'token': 12573,
  'token_str': 'freshwater',
  'sequence': 'mussel is a freshwater.'},
 {'score': 0.015674659982323647,
  'token': 3269,
  'token_str

In [55]:
import pandas as pd
df_result = pd.DataFrame()
df_result["hypernym"] = eval_parents
df_result["hyponym"] = eval_children

for column_name in tqdm(column_names):
    query_predictions = []

    for parent, child in zip(eval_parents, eval_children):
        preds = predictions[child][column_name]

        parent_in_preds = False

        for pred_list in preds:
            if parent == pred_list['token_str']:
                parent_in_preds = True
                break


        query_predictions.append(parent_in_preds * 1)

    df_result[column_name] = query_predictions

df_result['sum'] = df_result[column_names].sum(axis=1)
df_result

100%|██████████| 11/11 [00:00<00:00, 710.46it/s]


,hypernym,hyponym,{token} [MASK],[MASK] {token},is a,is an,kind of,type of,example of,such as,a such as,an such as,favorite,sum
0,seafood,mussel,0,0,0,0,0,0,0,1,1,0,1,3
1,seafood,clam,0,0,0,0,0,0,0,1,0,0,0,1
2,seafood,lobster,0,0,0,0,0,1,0,1,0,1,1,4
3,seafood,oyster,0,0,0,0,0,1,0,1,0,0,1,3
4,seafood,shrimp,0,0,0,0,0,1,0,1,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,music,jazz music,0,0,0,0,1,1,1,1,1,0,1,6
520,music,jazz,0,0,0,0,1,1,1,1,1,0,1,6
521,music,upbeat music,1,0,0,0,1,1,1,1,1,0,1,7
522,music,front desk,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
print(len(df_result.loc[df_result["sum"] > 0]) / len(df_result))

TypeError: '>' not supported between instances of 'method' and 'int'

In [ ]:
"""
Find the best matching guesses for the masked word
text: a sentence much include exactly one [MASK] token to predict.
      For example:
      'Alex likes to have [MASK] with his best friend'
model: a BertForMaskedLM
tokenizer: a Bert tokenizer
topn: Number of candidates for mask

Returns candidates and their probs
"""

# ref: ...

# def predict_word(text, model, tokenizer, topn=10):
def predict_word(text, model, tokenizer, distil=False):
    # Prepare tex
    text = '[CLS] '+ text.lstrip('[CLS] ').rstrip(' [SEP]')+' [SEP]'
    # Tokenize input
    tokenized_text = tokenizer.tokenize(text)

    # Mask a token that we will try to predict back with `BertForMaskedLM`
    masked_index = -1
    for i, token in enumerate(tokenized_text):
        if token=='[MASK]':
            masked_index = i
            break
    assert i>=0

    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
    segments_ids = [0]*len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # If you have a GPU, put everything on cuda
    tokens_tensor = tokens_tensor.to(device)
    if not distil:
        segments_tensors = segments_tensors.to(device)

    # Predict all tokens
    with torch.no_grad():
        if distil:
            outputs = model(tokens_tensor)
        else:
            outputs = model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0]
    # print("Predictions shape: " + str(predictions[0].shape))
    predicted_inds = torch.argsort(-predictions[0, masked_index])
    # print(predicted_inds[:topn])
    predicted_probs = [round(p.item(),4) for p in torch.softmax(predictions[0, masked_index], 0)[predicted_inds]]
    predicted_tokens = tokenizer.convert_ids_to_tokens([ind.item() for ind in predicted_inds])

    # return list(zip(predicted_tokens, predicted_probs))[:topn]
    return list(zip(predicted_tokens, predicted_probs)), dict(list(zip(predicted_tokens, predicted_probs)))

